<a href="https://colab.research.google.com/github/HaoranYueRyan/Data-Science-analysis-Approaches/blob/master/Copy_of_ML_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'aif360[LFR]'

In [ ]:
cd /usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult

/usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

--2022-04-26 17:35:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/x-httpd-php]
Saving to: ‘adult.data.45’

adult.data.45       100%[===================>]   3.79M  3.18MB/s    in 1.2s    

2022-04-26 17:35:35 (3.18 MB/s) - ‘adult.data.45’ saved [3974305/3974305]

--2022-04-26 17:35:35--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5229 (5.1K) [application/x-httpd-php]
Saving to: ‘adult.names.45’

adult.names.45      100%[===================>]   5.11K  --.-KB/s    in 0s    

In [ ]:
pip install fairlearn

In [ ]:
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
import fairlearn.reductions as red

privileged_groups=[{'sex':1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])


#STEP 3: We split between training and test set.
train_all, test_all = dataset_orig.split([0.7], shuffle=True)
print("training data size", train_all.features.shape)
print("dataset feature names", train_all.feature_names)

training data size (34189, 18)
dataset feature names ['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


In [ ]:
from sklearn.preprocessing import StandardScaler 


#Normalize the dataset, both train and test. This should always be done in any machine learning pipeline!
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(train_all.features)
y_train = train_all.labels.ravel()

X_test = scale_orig.transform(test_all.features) 
y_test = test_all.labels.ravel()

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True)
kf

KFold(n_splits=5, random_state=None, shuffle=True)

In [ ]:
def get_score(model,X_train,X_test,y_train,y_test):
  model.fit(X_train,y_train)
  return model.score(X_test,y_test)

In [ ]:
sum(y_train)

8261.0

In [ ]:
import pandas as pd 
import copy 
def new_metrix(dataset,y_prediction,unprivileged_groups, privileged_groups,index):
  dataset_pred=copy.deepcopy(dataset)
  y_prediction.resize((len(y_prediction),1))
  dataset_pred.labels[index]=y_prediction
  
  metric = ClassificationMetric(dataset, dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
  metric_arrs = {}
  metric_arrs['stat_par_diff']=(metric.statistical_parity_difference())
  metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
  metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
  # df=pd.DataFrame(metric_arrs,index[0,1,2])
  return metric_arrs
  # stat_par_diff.append(metric.statistical_parity_difference())
  # print(stat_par_diff)
  #     # metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
  #     # eq_opp_diff.append(metric.equal_opportunity_difference())
  #     # print(eq_opp_diff)
  # metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
  # avg_odds_diff.append(metric.average_odds_difference())
  #   # metric_arrs['mean_stat_par_diff'].append(mean(stat_par_diff))
  #   # print(mean(stat_par_diff))
  #   # metric_arrs['eq_opp_diff'].append(mean(eq_opp_diff))
  #   # metric_arrs['avg_odds_diff'].append(mean(avg_odds_diff))
  #   # metric_arrs['C_slover'].append(C_arr+'_'+slov)


In [ ]:
from aif360.metrics import ClassificationMetric
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
# data=train.copy()
score_mean=[]
stat_par_diff=[]
eq_opp_diff=[]
avg_odds_diff=[]
C_array_1=[1.0, 0.001, 0.000001]
C_array=[1.0, 0.001]
solver_array=['saga']
solver_array_1=['saga']

kernel_1=['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
kernel=['linear']
for ker in kernel:
  for C_arr in C_array:
    learner = SVC(gamma='auto',kernel=ker,C=C_arr)

    for train_index,test_index in kf.split(X_train):
      print('all')
      train_x,test_x,train_y,test_y=X_train[train_index].copy(),X_train[test_index].copy(),y_train[train_index].copy(),y_train[test_index].copy()
      # learner = SVC(gamma='auto',kernel=ker,C=C_arr)
      print(sum(train_y),'Y_label')
      
      learner.fit(train_x,train_y)
      predictions = learner.predict(test_x)
      
      print('try')
      data=train_all.copy()
      print(train_index.shape,'before')
      marix=new_metrix(data,predictions,unprivileged_groups, privileged_groups,test_index)
      print(train_index.shape,'after')
      print('X')
      # print(marix)

all
6599.0 Y_label
try
(27351,) before
(27351,) after
X
all
6571.0 Y_label
try
(27351,) before
(27351,) after
X
all
6606.0 Y_label
try
(27351,) before
(27351,) after
X
all
6606.0 Y_label
try
(27351,) before
(27351,) after
X
all
6662.0 Y_label
try
(27352,) before
(27352,) after
X
all
6565.0 Y_label
try
(27351,) before
(27351,) after
X
all
6623.0 Y_label
try
(27351,) before
(27351,) after
X
all
6591.0 Y_label
try
(27351,) before
(27351,) after
X
all
6647.0 Y_label
try
(27351,) before
(27351,) after
X
all
6618.0 Y_label
try
(27352,) before
(27352,) after
X


In [ ]:
def new_metrix_classifer(alpha_array,solver_array,X_train,y_train):
  for C_arr in alpha_array:

    for slov in solver_array:
      for train_index,test_index in kf.split(X_train):
        train_x,test_x,train_y,test_y=X_train[train_index],X_train[test_index],y_train[train_index],y_train[test_index]
        learner = MLPClassifier(alpha=C_arr, solver=slov)  
      # print(train_y[200:300],'1')
      # print(train_x[-1])
      
        learner.fit(train_x,train_y)
        predictions = learner.predict(test_x)
        marix=new_metrix(data,predictions,unprivileged_groups, privileged_groups)


In [ ]:
from aif360.metrics import ClassificationMetric
from sklearn.neural_network import MLPClassifier
score_mean=[]
stat_par_diff=[]
eq_opp_diff=[]
avg_odds_diff=[]
alpha_array=[1.0, 100000., 0.001, 0.000001]
solver_array=['lbfgs', 'sgd', 'adam']
metric_arrs = {}
for C_arr in alpha_array:
  for slov in solver_array:
    

    for train_index,test_index in kf.split(X_train):
      
      train_x,test_x,train_y,test_y=X_train[train_index],X_train[test_index],y_train[train_index],y_train[test_index]
      learner = MLPClassifier(alpha=C_arr, solver=slov)  
      print(train_y[200:300],'1')
      print(train_x[-1])
      
      learner.fit(train_x,train_y)
      predictions = learner.predict(test_x)

  # print(predictions.shape)
  
      train_pred=train.copy()
      predictions.resize((len(predictions),1))
      train_array=train_pred.labels
      train_array[test_index]=predictions

      metric = ClassificationMetric(train, train_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
      metric_arrs = {}
      metric_arrs['stat_par_diff']=(metric.statistical_parity_difference())
      stat_par_diff.append(metric.statistical_parity_difference())
      print(stat_par_diff)
      # metric_arrs['eq_opp_diff']=(metric.equal_opportunity_difference())
      # eq_opp_diff.append(metric.equal_opportunity_difference())
      # print(eq_opp_diff)
      metric_arrs['avg_odds_diff']=(metric.average_odds_difference())
      avg_odds_diff.append(metric.average_odds_difference())
    # metric_arrs['mean_stat_par_diff'].append(mean(stat_par_diff))
    # print(mean(stat_par_diff))
    # metric_arrs['eq_opp_diff'].append(mean(eq_opp_diff))
    # metric_arrs['avg_odds_diff'].append(mean(avg_odds_diff))
    # metric_arrs['C_slover'].append(C_arr+'_'+slov)

df=pd.DataFrame(metric_arrs)
print(metric_arrs)

  # print(metric_arrs)
  # print('\n')

[0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 1. 0. 1.] 1
[ 0.41417173 -1.42511042 -0.2314253  -0.57157476 -0.60216288 -0.52874978
 -0.39611848  3.87764314 -0.14404855 -0.16988074 -0.19633643 -0.11418154
 -0.69030989 -0.5361253  -0.20976817  5.4488811  -0.232755   -0.57711385]


NameError: ignored